In [23]:
import pandas as pd
import numpy as np
from pprint import pprint

In [24]:
dataset=pd.read_csv('source/zoo.csv',
                      names=['animal_name','hair','feathers','eggs','milk',
                     'airbone','aquatic','predator','toothed','backbone',
                     'breathes','venomous','fins','legs','tail','domestic','catsize','class',])

In [25]:
dataset=dataset.drop('animal_name',axis=1)

In [26]:
def entropy(target_col):
    """
    Calculate the entropy of a dataset.
    The only parameter of this function is the target_col parameter which specifies the target column
    """
    elements, counts = np.unique(target_col, return_counts = True)
    print(elements,counts)
    entropy=-np.sum([(counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    
# Please implement the routine to compute the entropy given the data
    # 

    return entropy


In [27]:
def InfoGain(data, split_attribute_name, target_name):
    #전체 엔트로피 계산 = 참가여부
    total_entropy = entropy(data[target_name])
    
    #가중 엔트로피 계산
    vals,counts = np.unique(data[split_attribute_name],return_counts=True)
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*
                               entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name])
                               for i in range(len(vals))])
    print(split_attribute_name, '=', round(Weighted_Entropy,5))
    
    #정보 획득량 계산
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain

In [28]:
def ID3(data, originaldata, features,target_attribute_name="class", parent_node_class = None):
   
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    
    #If the dataset is empty, return the mode target feature value in the original dataset
    elif len(data) == 0:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],return_counts=True)[1])]
    
    #If the feature space is empty, return the mode target feature value of the direct parent node --> Note that
    #the direct parent node is that node which has called the current run of the ID3 algorithm and hence
    #the mode target feature value is stored in the parent_node_class variable.
    
    elif len(features) == 0:
        return parent_node_class
    
    #If none of the above holds true, grow the tree!
    
    else:
        #Set the default value for this node --> The mode target feature value of the current node
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]
        
        #Select the feature which best splits the dataset
        item_values = [InfoGain(data, feature, target_attribute_name) for feature in features] #Return the information gain values for the features in the dataset
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        
        #Create the tree structure. The root gets the name of the feature (best_feature) with the maximum information
        #gain in the first run
        tree = {best_feature:{}}
        
        
        #Remove the feature with the best inforamtion gain from the feature space
        features = [i for i in features if i != best_feature]
        
        #Grow a branch under the root node for each possible value of the root node feature
        
        for value in np.unique(data[best_feature]):
            value = value
            #Split the dataset along the value of the feature with the largest information gain and therwith create sub_datasets
            sub_data = data.where(data[best_feature] == value).dropna()
            
            #Call the ID3 algorithm for each of those sub_datasets with the new parameters --> Here the recursion comes in!
            subtree = ID3(sub_data,dataset,features,target_attribute_name,parent_node_class)
            
            #Add the sub tree, grown from the sub_dataset to the tree under the root node
            tree[best_feature][value] = subtree
            
        return(tree)    
                
###################

###################
    
def predict(query,tree,default = 1):

    #1.
    for key in list(query.keys()):
        if key in list(tree.keys()):
            #2.
            try:
                result = tree[key][query[key]] 
            except:
                return default
  
            #3.
            result = tree[key][query[key]]
            #4.
            if isinstance(result,dict):
                return predict(query, result)

            else:
                return result
        

def train_test_split(dataset):
    training_data = dataset.iloc[:80].reset_index(drop=True)#We drop the index respectively relabel the index
    #starting form 0, because we do not want to run into errors regarding the row labels / indexes
    testing_data = dataset.iloc[80:].reset_index(drop=True)
    return training_data,testing_data

training_data = train_test_split(dataset)[0]
testing_data = train_test_split(dataset)[1] 

def test(data,tree):
    #Create new query instances by simply removing the target feature column from the original dataset and 
    #convert it to a dictionary
    queries = data.iloc[:,:-1].to_dict(orient = "records")
    
    #Create a empty DataFrame in whose columns the prediction of the tree are stored
    predicted = pd.DataFrame(columns=["predicted"]) 
    
    #Calculate the prediction accuracy
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i],tree,1.0) 
    print('The prediction accuracy is: ',(np.sum(predicted["predicted"] == data["class"])/len(data))*100,'%')
    
"""
Train the tree, Print the tree and predict the accuracy
"""
def build_decision_tree():

    tree = ID3(training_data,training_data,training_data.columns[:-1])
    
    pprint(tree)
    
    test(testing_data,tree)
    
if __name__ == '__main__':
	build_decision_tree()

[1 2 3 4 5 6 7] [36 16  2 10  3  6  7]
[1. 2. 3. 4. 5. 6. 7.] [ 2 16  2 10  3  3  7]
[1. 6.] [34  3]
hair = 1.47462
[1 2 3 4 5 6 7] [36 16  2 10  3  6  7]
[1. 3. 4. 5. 6. 7.] [36  2 10  3  6  7]
[2.] [16]
feathers = 1.53434
[1 2 3 4 5 6 7] [36 16  2 10  3  6  7]
[1. 3. 7.] [35  1  1]
[1. 2. 3. 4. 5. 6. 7.] [ 1 16  1 10  3  6  6]
eggs = 1.41951
[1 2 3 4 5 6 7] [36 16  2 10  3  6  7]
[2. 3. 4. 5. 6. 7.] [16  2 10  3  6  7]
[1.] [36]
milk = 1.26349
[1 2 3 4 5 6 7] [36 16  2 10  3  6  7]
[1. 2. 3. 4. 5. 6. 7.] [35  4  2 10  3  1  7]
[1. 2. 6.] [ 1 12  5]
airbone = 1.78049
[1 2 3 4 5 6 7] [36 16  2 10  3  6  7]
[1. 2. 3. 6. 7.] [30 11  1  6  2]
[1. 2. 3. 4. 5. 7.] [ 6  5  1 10  3  5]
aquatic = 1.87408
[1 2 3 4 5 6 7] [36 16  2 10  3  6  7]
[1. 2. 4. 6.] [15  8  3  5]
[1. 2. 3. 4. 5. 6. 7.] [21  8  2  7  3  1  7]
predator = 2.09296
[1 2 3 4 5 6 7] [36 16  2 10  3  6  7]
[1. 2. 6. 7.] [ 1 16  6  7]
[1. 3. 4. 5.] [35  2 10  3]
toothed = 1.38424
[1 2 3 4 5 6 7] [36 16  2 10  3  6  7]
[6. 7.] [6